In [61]:
import torch
import numpy as np
from NN_model import *

### Load the model

In [62]:
device = torch.device('cpu' if torch.cuda.is_available() else 'cpu')
model = Classifier(input_dim=6)
model.load_state_dict(torch.load('model.pt'))
model = model.to(device)

### Get dSpace data

In [63]:
# file = r"C:\Users\20182152\Documents\Omron_project\5ARIP10\HD_data\dSpace\F1_dSpace_test_3_speed_30_sec60.csv"

file = "Demo2.csv"

speed = 30
split_time = 2 # Time of sample in secs
cur_samplerate = 1000

arr = np.loadtxt(file, delimiter=",", dtype=str, usecols=(1,2,3,4,5,6), skiprows=16)
data = arr[12:] # data consists of ['time','I_bat[A]', 'I_dyno_LP[A]', 'i_motor_LP[A]','speed_motor[rads]', 'u_bat[V]']
data = data.astype(float)

cur_data = data[:,3] #Only obtain current running through motor
cur_data = cur_data[:round(split_time * cur_samplerate)]


### Get statistical features from measurement

In [64]:
mean_cur = np.float32(np.mean(cur_data))
stds_cur = np.float32(np.std(cur_data))
maxs_cur = np.float32(np.max(cur_data))
mins_cur = np.float32(np.min(cur_data))
medians_cur = np.float32(np.median(cur_data))

inputs = torch.tensor([speed, mean_cur, stds_cur, maxs_cur, mins_cur, medians_cur])

### Make prediction

In [65]:

with torch.no_grad():
  pred = model(inputs)

predicted = (pred >= 0.5).squeeze().long()

if predicted == 1:
  confidence = pred.item() * 100
  print(f"Drive is anomalous, confidence: {confidence}%")
else:
  confidence = (1-pred.item()) * 100
  print(f"Drive is normal, confidence: {confidence}%")


Drive is normal, confidence: 100.0%
